In [1]:
# importing libraries
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from nexinfosys import NISClient, display_visjs_jupyterlab
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from nexinfosys import NISClient, display_visjs_jupyterlab
import io
import networkx as nx
import csv
import qgrid



In [2]:
def write_csv(output_csv,name): #eg flow_graph_solution.csv
    with open(name,'wb') as file:
        for line in output_csv:
            file.write(line)
    return name

In [3]:
# Input Nis Format File
#fname =  'howTo Clone CloneScale.xlsx'
fname = 'howTo Indicators.xlsx'
#fname = 'howTo Examples ScaleChange-Observers-Conflict.xlsx'

In [4]:
!ls

EDA-Notebook.ipynb			   howTo Indicators.xlsx
howTo Clone CloneScale.xlsx		   README.md
howTo Examples ScaleChange-Observers.xlsx


### Connect to Nis Client

In [5]:
#c = NISClient("https://one.nis.magic-nexus.eu/nis_api")
#To local:
#c = NISClient("http://localhost:5000/nis_api")
c = NISClient("http://172.17.0.1:5000/nis_api")
# Login, open session, load a workbook (which is in Nextcloud), submit (execute!)
c.login("test_user")
#print("Logged in")

True

In [6]:
c.open_session()
#print("Session opened")

True

In [7]:
n = c.load_workbook(fname)
print("N worksheets: "+str(n))

N worksheets: 10


In [8]:
r = c.submit()

In [9]:
any_error = False
if len(r) > 0:
    for i in r:
        if i["type"] == 3:
            any_error = True
            print(str(i))
            

In [10]:
if not any_error:
    # Obtain available datasets
    r = c.query_available_datasets()
    if len(r) > 0:
        results = {}
        for ds in r:
            results[ds["name"]] = {d["format"].lower(): d["url"] for d in ds["formats"]}


In [11]:
#See Warnings
#pd.DataFrame(r)

### Available Resuts

In [12]:
outputs =pd.DataFrame(c.query_available_datasets())
outputs

,description,formats,name,type
0,"Parameter values per Scenario [4 rows, 12 cell...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",params,dataset
1,"Flow Graph Solver - Interfaces [160 rows, 2880...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution,dataset
2,"Flow Graph Solver Edges - Interfaces [66 rows,...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution_edges,dataset
3,"Flow Graph Solver - Local Indicators [50 rows,...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution_indicators,dataset
4,"Flow Graph Solver - Global Indicators [2 rows,...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_global_indicators,dataset
5,"Flow Graph Solver - Local Benchmarks [50 rows,...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution_benchmarks,dataset
6,"Flow Graph Solver - Global Benchmarks [2 rows,...","[{'format': 'CSV', 'url': '/nis_api/isession/r...",flow_graph_solution_global_benchmarks,dataset
7,"None [15 rows, 300 cells, 2480 bytes]","[{'format': 'CSV', 'url': '/nis_api/isession/r...",Matrix1,dataset
8,"None [14 rows, 252 cells, 2096 bytes]","[{'format': 'CSV', 'url': '/nis_api/isession/r...",Matrix2,dataset
9,"Graph of Interfaces, Quantities; Scales and Ex...","[{'format': 'VisJS', 'url': '/nis_api/isession...",interfaces_graph,graph


In [13]:
listoutputs = outputs['name']

## Download some result

In [14]:
def download(c):
    outputs =pd.DataFrame(c.query_available_datasets())
    listoutputs = outputs['name']

    flow_graph_solution = c.download_results([(results["flow_graph_solution"]['csv'])])
    write_csv(flow_graph_solution,'flow_graph_solution.csv')

    if any(listoutputs == 'flow_graph_global_indicators'):
        flow_graph_global_indicators = c.download_results([(results["flow_graph_global_indicators"]['csv'])])
        write_csv(flow_graph_global_indicators,'flow_graph_global_indicators.csv')

    if any(listoutputs == 'flow_graph_solution_indicators'):
        flow_graph_solution_indicators = c.download_results([(results["flow_graph_solution_indicators"]['csv'])])
        write_csv(flow_graph_solution_indicators,'flow_graph_solution_indicators.csv')

    processors_geolayer = c.download_results([(results["processors_geolayer"]['geojson'])]) 
    write_csv(processors_geolayer,'processors_geolayer.geojson')

    flow_graph_solution_edges = c.download_results([(results["flow_graph_solution_edges"]['csv'])]) 
    write_csv(flow_graph_solution_edges,'flow_graph_solution_edges.csv')
    

In [15]:
processors_graph = c.download_results([(results["processors_graph"]["visjs"])])
interfaces_graph = c.download_results([(results["interfaces_graph"]["visjs"])])
download(c)

In [16]:
c.check_backend_available()

True

###  Terminate Session

In [17]:
c.close_session()
c.logout()

True

## Processors Graph

In [18]:
def display_visjs(graph):
    visjs_data2 = graph[0].decode("utf-8")
    un2 = None
    #unique_name = display_visjs_jupyterlab(visjs_data, 800, unique_name)
    un2 = display_visjs_jupyterlab(visjs_data2, 1200, un2)


In [19]:
#Processors
display_visjs(processors_graph)

<IPython.core.display.Javascript object>

In [20]:
#Interfaces
display_visjs(interfaces_graph)

<IPython.core.display.Javascript object>

## Exploring Flow Graph Solution

In [21]:
FGSds = pd.read_csv("flow_graph_solution.csv")

In [22]:
%%bash
jupyter nbextension enable --py --sys-prefix qgrid

Enabling notebook extension qgrid/extension...
      - Validating: OK


In [23]:
qgrid_widget = qgrid.show_grid(FGSds,show_toolbar= True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Data Clonflict

In [24]:
def ConflictStudy(df,conflictType): 
    if any(df[conflictType] =='Dismissed'):
        df = df.drop(columns=['Expression','Computed','Observer'])
        Taken = df[df[conflictType]=='Taken'].drop([conflictType], axis= 1)
        Dismissed = df[df[conflictType]=='Dismissed'].drop([conflictType], axis= 1)
        Taken = Taken.rename(columns={'Value':'Value_Taken'})
        Dismissed = Dismissed.rename(columns={'Value':'Value_Dismissed'})
        Conflict = pd.merge(Taken,Dismissed, how = 'outer')
        Conflict['diff'] = Conflict['Value_Taken'] - Conflict['Value_Dismissed']
        return Conflict
    else:
        print('No Conflicts found')

#### Part Of Conflict

In [25]:
ConflictItype = ConflictStudy(FGSds,'Conflict_Itype')

No Conflicts found


In [26]:
ConflictStudy(FGSds,'Conflict_Partof')


,Scenario,Period,Scope,Conflict_Itype,Processor,Interface,Orientation,RoegenType,Value_Taken,Unit,Level,System,Subsystem,Sphere,Value_Dismissed,diff
0,default,2020,External,No,France.crop_production,wheat,Output,Flow,100.0,tonnes,n-1,SP,External,Technosphere,100.0,0.0
1,default,2020,Internal,No,Spain.crop_production,wheat,Output,Flow,100.0,tonnes,n-1,SP,Local,Technosphere,100.0,0.0
2,default,2020,Total,No,France.crop_production,wheat,Output,Flow,100.0,tonnes,n-1,SP,External,Technosphere,100.0,0.0
3,default,2020,Total,No,Spain.crop_production,wheat,Output,Flow,100.0,tonnes,n-1,SP,Local,Technosphere,100.0,0.0


### Flow graph solution Study

#### Absolute Values Mode

In [108]:
def cleanFGS(df):
    df = df[(df['Conflict_Partof'] != 'Dissmissed')&(df['Conflict_Itype']!='Dismissed')]
    #Simplified dataFrame for accounting exploration
    df = df.drop(columns= ['Conflict_Partof','Conflict_Itype', 'Computed','Observer','Expression'])
    # los observers cambian según el escenario entonces en realidad daría igual si dejarlo o no en el dataframe??

    Inputs = df[df['Orientation']=='Input']
    Outputs = df[df['Orientation'] == 'Output']

    Inputs =Inputs.rename(columns= {'Value': 'Value_Input'})
    Outputs = Outputs.rename(columns={'Value':'Value_Output'})

    Inputs = Inputs.drop(columns = "Orientation")
    Outputs = Outputs.drop(columns = "Orientation")

    data = pd.merge(Inputs,Outputs, how = 'outer')

    data['Value'] = abs(data['Value_Input'].fillna(0)-data['Value_Output'].fillna(0))
    # Replace Level nan in subsystem processor for processor name ! if you have not defined processors all studies bellow will be done by processor instead of Level 
    data = data.fillna("")
    SubsystemProcessor = data[data['Level']== ""]['Processor']
    data.loc[data['Level'] == "", 'Level'] = SubsystemProcessor
    #Store
    data.to_csv('flow_graph_solution_clean.csv')

In [109]:
cleanFGS(FGSds)

In [110]:
FGSds2 = pd.read_csv('flow_graph_solution_clean.csv')

In [111]:
# Ver porqué no me funciona esto
qgrid_widget = qgrid.show_grid(FGSds2,show_toolbar= True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Dar la opción de ver input y output o de verlo en total en la gráfica de barras poner el input como +y aoutput como negativo?

### BarChart by Level

In [127]:
fig = go.FigureWidget( layout=go.Layout())
@interact
def Bar_chart_Level(Level = list(set(FGSds2['Level'])),Scenario = list(set(FGSds2['Scenario'])),Period = list(set(FGSds2['Period'])),Scope = list(set(FGSds2['Scope'])),Interface = list(set(FGSds2['Interface']))):
    data = pd.read_csv("flow_graph_solution_clean.csv")
    data = data[(data['Scenario'] == Scenario)&(data['Level'] == Level)&(data['Period'] == Period)&(data['Scope'] == Scope)&( data['Interface'] == Interface)]    
    y_pos =np.arange(len(data.Processor))
    plt.bar(y_pos,data['Value'])
    plt.xticks(y_pos,data.Processor, rotation = 90)
    plt.show()

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)   


interactive(children=(Dropdown(description='Level', options=('n-2', 'SP_Environment', 'n-1', 'SP_Local', 'n'),…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [121]:

fig = go.FigureWidget( layout=go.Layout())
@interact
def Bar_chart_scenario(Scope = list(set(FGSds2['Scope'])),
    Interface = list(set(FGSds2['Interface'])),
    Processor = list(set(FGSds2['Processor'])),
    Period = list(set(FGSds2['Period']))):
    
    data = pd.read_csv("flow_graph_solution_clean.csv")
    data = data[(data['Processor'] == Processor)&(data['Period'] == Period)&(data['Scope'] == Scope)&(data['Interface'] == Interface)]    
    y_pos =np.arange(len(data.Scenario))
    plt.bar(y_pos,data.Value)
    plt.xticks(y_pos,data.Scenario, rotation = 90)
    plt.show()

interactive(children=(Dropdown(description='Scope', options=('Internal', 'External', 'Total'), value='Internal…

In [122]:
fig = go.FigureWidget( layout=go.Layout())
@interact
def Time_Interface(Scope = list(set(FGSds2['Scope'])),
          Interface = list(set(FGSds2['Interface'])),
          Scenario = list(set(FGSds2['Scenario'])),
          Level = list(set(FGSds2['Level']))):
    data = pd.read_csv("flow_graph_solution_clean.csv")
    data = data[(data['Level'] == Level)&(data['Scope'] == Scope)&(data['Interface'] == Interface)&(data['Scenario']== Scenario)]  
    Processors= list(set(data['Processor']))
    Unit = list(set(data['Unit']))  
    datatempspread =  data[['Processor','Value','Period']].pivot(index = 'Period',columns='Processor',values='Value')
    datatempspread = datatempspread.reset_index()
    fig, ax = plt.subplots()
    for nm in Processors:
        ax.plot(datatempspread['Period'],datatempspread[nm], label=nm)
        # Set x-ticks to yearly interval, adjust y-axis limits, add legend and labels   
        ax.legend()
        ax.set_ylabel(Interface[0]+' ' + Unit[0])
        ax.set_title(Level);
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.show()

interactive(children=(Dropdown(description='Scope', options=('Internal', 'External', 'Total'), value='Internal…

In [ ]:
FGSed = pd.read_csv('flow_graph_solution_edges.csv')

In [ ]:
qgrid_widget = qgrid.show_grid(FGSed,show_toolbar= True)
qgrid_widget

## Indicators

In [126]:
FGSind = pd.read_csv('flow_graph_solution_indicators.csv').fillna("")
qgrid_widget = qgrid.show_grid(FGSind,show_toolbar= True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…